<a href="https://colab.research.google.com/github/terence-mitterrand/terence-mitterrand-DataScience-GenAI-Submissions/blob/main/Exercise_2/2_02_Joining_DataFrames_COMPLTED.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![](https://drive.google.com/uc?export=view&id=1xqQczl0FG-qtNA2_WQYuWePW9oU8irqJ)

# 2.02 Pandas: Joining and merging dataframes
### Why Combine Dataframes?
Quite often in practical applications we have data coming from multiple sources such as different files, database tables or IoT devices. This is particularly likely when working in data science and artificial intelligence. However, typically we want to create a single dataframe to run our analysis and/or models. To achieve this commonly we would want to combine dataframes together.

Joining data is a large topic and an important one in database administration and data engineering. We will look at this at a relatively high level, althout the interested reader can certainly explore this in much more depth.

Let us begin by using the orders_df from the previous session.

In [ ]:
import pandas as pd
import numpy as np

orders = {'o10001':{'date':'2024/01/10', 'product':'Hoodie', 'quantity':'1'},
            'o10002':{'date':'2024/01/13', 'product':'Tote bag', 'quantity':'2'},
            'o10003':{'date':'2024/01/14', 'product':'Pencil', 'quantity':'10'},
            'o10004':{'date':'2024/01/15', 'product':'T-shirt', 'quantity':'2'}
}

orders_df = pd.DataFrame(orders)
orders_df

,o10001,o10002,o10003,o10004
date,2024/01/10,2024/01/13,2024/01/14,2024/01/15
product,Hoodie,Tote bag,Pencil,T-shirt
quantity,1,2,10,2


In this scenario, we also have some data about these products from our operations team. The data is currently in a dictionary and can be transformed in the usual fashion:

In [ ]:
products = {'123':{'name':'Hoodie', 'cost_price':12.12, 'sale_price':15.00},
            '124':{'name':'Tote bag', 'cost_price':2.15, 'sale_price':9.99},
            '125':{'name':'Pencil', 'cost_price':22.45, 'sale_price':49.99},
            '126':{'name':'T-shirt', 'cost_price':0.45, 'sale_price':2.99},
            '127':{'name':'Honours degree', 'cost_price':0.78, 'sale_price':1.49}
}

products_df = pd.DataFrame(products)
products_df

,123,124,125,126,127
name,Hoodie,Tote bag,Pencil,T-shirt,Honours degree
cost_price,12.12,2.15,22.45,0.45,0.78
sale_price,15.0,9.99,49.99,2.99,1.49


### Joining Dataframes using Merge
What we want to do is join these datasets together. However, currently the Dataframes are both the wrong way round. We would want our rows in orders_df to be the order ID, and the rows in products_df to be the product ID. We can transform the data using the Dataframe.transpose function:

In [ ]:
orders_df = orders_df.transpose()
products_df = products_df.transpose()

Transpose basically flips the axes of the Dataframe so the current columns become rows and vice versa. Note, _Dataframe.transpose_ creates a new object by default. We use the "orders_df = ..." element at the start to reassign the variable and make orders_df equal the new, transposed version.

In [ ]:
orders_df

,date,product,quantity
o10001,2024/01/10,Hoodie,1
o10002,2024/01/13,Tote bag,2
o10003,2024/01/14,Pencil,10
o10004,2024/01/15,T-shirt,2


With this in place we can join the two Dataframes. There are several decisions that need to be taken when joining two datasets. Firstly, there are multiple ways to join data in pandas but the most appropriate here would seem to be using the _merge( )_ function. These choices will be discussed in more detail later in the notebook. Secondly we need to decide on the element that we will make the join on. In this case, the data that the two dataframes have in common are the product name, labelled “product” in orders_df and as “name” in products_df. We also need to decide the type of join – what do we want to do with rows that are in one set but not in the other? This too is discussed further below. In this case the scenario is to get more details on our orders, so we are joining products_df to orders_df so we will base our rows on the orders_df dataframe. With these choices in mind we get the following code:

In [ ]:
joined_df = orders_df.merge(products_df, how='left', left_on='product', right_on='name')

We’ll unpack this a little more because it is a more complex function. We are creating a new dataframe called joined_df. This will be based on the orders_df being merged with products_df as a ‘left join’. This means we will use the rows of the orders_df not products_df (as orders_df is on the left hand of the command). Finally we need to tell Python which columns we will be joining based on. In this case the left hand dataframe (orders_df) lists the relevant data as “product”, and the right hand (products_df) lists it as “name”.

In [ ]:
joined_df

,date,product,quantity,name,cost_price,sale_price
0,2024/01/10,Hoodie,1,Hoodie,12.12,15.0
1,2024/01/13,Tote bag,2,Tote bag,2.15,9.99
2,2024/01/14,Pencil,10,Pencil,22.45,49.99
3,2024/01/15,T-shirt,2,T-shirt,0.45,2.99


There are several types of joins that can be used here. The main types are shown below. The yellow bit in each Venn diagram represents the bit we would keep if we perform this type of join.

So far we have only looked at a "left join" (labelled in pandas as "left"). It would also be possible to specify an "inner join" ("inner"). This means we only keep records (rows) that are in both datasets. Only the records in both would be included if we use an "inner join". Finally we could use a "full join" ("outer") which means that we keep all the records irrespective of whether they are missing in one of the sides.

![](https://drive.google.com/uc?export=view&id=1e8rPwaCqpnrB9-yIcqKOm2sb6VUPGz_g)

Let’s return to our first merged dataframe (joined_df) as this better fits the description of this problem (we care more about orders in this problem than products). One immediate issue is that we have duplicated columns product and name. Let’s tidy this up by dropping the name column:

In [ ]:
joined_df = joined_df.drop(['name'], axis=1)
joined_df

,date,product,quantity,cost_price,sale_price
0,2024/01/10,Hoodie,1,12.12,15.0
1,2024/01/13,Tote bag,2,2.15,9.99
2,2024/01/14,Pencil,10,22.45,49.99
3,2024/01/15,T-shirt,2,0.45,2.99


The _drop( )_ function can be used to remove columns or rows. axis=1 indicates that we want to a drop a column(s), axis=0 would mean we were dropping a row(s). Note as well that although we are only dropping one column we still need to present the column name inside a list as this is what the function expects (a list).

### EXERCISE
In the above example we merged using a left join. How would different types of joins effect the final dataframe? Experiment with inner join, right join and outer join. What impact does it have on the data retained? Are the results what you expected them to be?

To start with I will ask gemini to create two simple dictionaries.

In [ ]:
dict1 = {
    'id': [1, 2, 3, 4],
    'name': ['Alice', 'Bob', 'Charlie', 'David'],
    'city': ['New York', 'London', 'Paris', 'Tokyo']
}

dict2 = {
    'id': [3, 4, 5, 6],
    'age': [30, 25, 35, 40],
    'occupation': ['Engineer', 'Artist', 'Doctor', 'Developer']
}

print("Dictionary 1:")
print(dict1)
print("\nDictionary 2:")
print(dict2)

Dictionary 1:
{'id': [1, 2, 3, 4], 'name': ['Alice', 'Bob', 'Charlie', 'David'], 'city': ['New York', 'London', 'Paris', 'Tokyo']}

Dictionary 2:
{'id': [3, 4, 5, 6], 'age': [30, 25, 35, 40], 'occupation': ['Engineer', 'Artist', 'Doctor', 'Developer']}


Now I will ask gemini to create two dataframes from the dictionaries.

In [ ]:
df1 = pd.DataFrame(dict1)
df2 = pd.DataFrame(dict2)

print("DataFrame 1:")
display(df1)

print("\nDataFrame 2:")
display(df2)

DataFrame 1:


,id,name,city
0,1,Alice,New York
1,2,Bob,London
2,3,Charlie,Paris
3,4,David,Tokyo



DataFrame 2:


,id,age,occupation
0,3,30,Engineer
1,4,25,Artist
2,5,35,Doctor
3,6,40,Developer


Now I wil perform an inner join to see what data is kept. I expect that only the rows of ID 3 and 4 will be kept with more detail as the variables from both datasets will be joined.

In [ ]:
inner_joined_df = pd.merge(df1, df2, on='id', how='inner')
display(inner_joined_df)

,id,name,city,age,occupation
0,3,Charlie,Paris,30,Engineer
1,4,David,Tokyo,25,Artist


As expected the output is 2 row for id 3 and 4 but with 4 variables.

Now I will experiment with right join.

In [ ]:
right_joined_df = pd.merge(df1, df2, on='id', how='right')
display(right_joined_df)

,id,name,city,age,occupation
0,3,Charlie,Paris,30,Engineer
1,4,David,Tokyo,25,Artist
2,5,NaN,NaN,35,Doctor
3,6,NaN,NaN,40,Developer


Here I can see that the joined dataframe includes all rows from the second dataset and the columns of both datasets however there as missing values for id 5 and 6 because the are only present in the second dataframe.

Now I will experiment with full join. I expected all columns and row to be included in the joined dataframe.

In [ ]:
full_joined_df = pd.merge(df1, df2, on='id', how='outer')
display(full_joined_df)

,id,name,city,age,occupation
0,1,Alice,New York,NaN,NaN
1,2,Bob,London,NaN,NaN
2,3,Charlie,Paris,30.0,Engineer
3,4,David,Tokyo,25.0,Artist
4,5,NaN,NaN,35.0,Doctor
5,6,NaN,NaN,40.0,Developer


Here we can see that all rows and columns of both datasets are included, as expected. However there are missing values for id 1 and 2 since they are only in the first dataframe and hence there is no match with the columns of the second.